In [32]:
import time
import random
from collections import defaultdict
import numpy as np

# Variables
exams = ['Matemática', 'Física', 'Química', 'Historia', 'Inglés', 'Biología', 'Computación']
days = ['Lunes', 'Martes', 'Miércoles']

# Dominio inicial
domain = {exam: days[:] for exam in exams}

# Cursos por estudiante
students = {
    'Ana': ['Matemática', 'Física', 'Inglés'],
    'Luis': ['Física', 'Historia', 'Computación'],
    'Sofía': ['Matemática', 'Química', 'Biología'],
    'Carlos': ['Historia', 'Biología', 'Inglés']
}

# Relación: cursos que no deben ser el mismo día si los comparten estudiantes
def conflict(c1, c2):
    for cursos in students.values():
        if c1 in cursos and c2 in cursos:
            return True
    return False

# Generar pares de cursos conflictivos
conflict_pairs = set()
for i in range(len(exams)):
    for j in range(i + 1, len(exams)):
        if conflict(exams[i], exams[j]):
            conflict_pairs.add((exams[i], exams[j]))


## Backtracking Search

In [33]:
def is_valid(assignment, var, value):
    for other_var, other_val in assignment.items():
        if value == other_val and ((var, other_var) in conflict_pairs or (other_var, var) in conflict_pairs):
            return False
    return True


def backtracking(assignment):
    if len(assignment) == len(exams):
        return assignment

    unassigned = [v for v in exams if v not in assignment]
    var = unassigned[0]
    for value in days:
        if is_valid(assignment, var, value):
            assignment[var] = value
            result = backtracking(assignment)
            if result:
                return result
            del assignment[var]
    return None

# Ejecutar y medir tiempo
start_time = time.time()
backtracking_result = backtracking({})
end_time = time.time()
backtracking_time = end_time - start_time

print("Backtracking Result:", backtracking_result)
print("Execution Time:", end_time - start_time, "seconds")


Backtracking Result: {'Matemática': 'Lunes', 'Física': 'Martes', 'Química': 'Miércoles', 'Historia': 'Lunes', 'Inglés': 'Miércoles', 'Biología': 'Martes', 'Computación': 'Miércoles'}
Execution Time: 0.0005128383636474609 seconds


## Beam Search

In [ ]:
def evaluate(state):
    """Evalúa cuántos conflictos tiene el estado actual"""
    conflicts = 0
    for (a, b) in conflict_pairs:
        if a in state and b in state and state[a] == state[b]:
            conflicts += 1
    return conflicts

def beam_search(k=3):
    beam = [{}]  # Lista de posibles soluciones parciales
    while True:
        new_beam = []
        for partial in beam:
            unassigned = [v for v in exams if v not in partial]
            if not unassigned:
                new_beam.append(partial)
                continue
            var = unassigned[0]
            for day in days:
                new_assignment = partial.copy()
                new_assignment[var] = day
                new_beam.append(new_assignment)

        # Evaluar todos y quedarse con los k mejores
        new_beam = sorted(new_beam, key=lambda x: evaluate(x))[:k]
        # Si todos están completos y sin conflictos, retornar
        for candidate in new_beam:
            if len(candidate) == len(exams) and evaluate(candidate) == 0:
                return candidate
        beam = new_beam

start_time = time.time()
beam_result = beam_search(k=3)
end_time = time.time()

print("Beam Search Result:", beam_result)
print("Execution Time:", end_time - start_time, "seconds")